In [17]:
from fealpy.mesh.triangle_mesh import TriangleMesh
from fealpy.functionspace import LagrangeFESpace
from fealpy.fem.diffusion_integrator import DiffusionIntegrator
from fealpy.fem.scalar_source_integrator import ScalarSourceIntegrator
from fealpy.fem.bilinear_form import BilinearForm
from fealpy.fem.linear_form import LinearForm   

from scipy.sparse.linalg import spsolve
from scipy.sparse import spdiags

from poisson_2d import CosCosData

import numpy as np

mesh = TriangleMesh.from_box(box=[0, 1, 0, 1], nx=3, ny=3)
pde = CosCosData()
p = 1
space = LagrangeFESpace(mesh, p=p, spacetype='C', doforder='vdims')
NDof = space.number_of_global_dofs()

# Assemble the stiffness matrix
bform = BilinearForm(space)
bform.add_domain_integrator(DiffusionIntegrator(q=p+2))
A0 = bform.assembly()

# Assemble the load vector
lform = LinearForm(space)
lform.add_domain_integrator(ScalarSourceIntegrator(pde.source, q=p+2))
F0 = lform.assembly()

# 纯 Dirichlet 边界条件处理有两种方式

把 $F$ 中 Dirichlet 边界自由度对应的分量设真解的值，并把 $A$ 中 Dirichlet 边界自由度对应的行列的主对角元素改为 $1$，其他元素改为 $0$（这种方法可以保证矩阵 $A$ 是一个对称矩阵）

In [18]:
# Apply Dirichlet boundary conditions
uh1 = space.function()
isBdDof = space.is_boundary_dof()
ipoints = space.interpolation_points() 
uh1[isBdDof] = pde.dirichlet(ipoints[isBdDof])

F1 = F0 - A0@uh1 
F1[isBdDof] = uh1[isBdDof]

bdIdx = np.zeros(NDof, dtype=np.int_)
bdIdx[isBdDof] = 1
Tbd = spdiags(bdIdx, 0, NDof, NDof) 
T = spdiags(1-bdIdx, 0, NDof, NDof)
A1 = T@A0@T + Tbd
A1_is_symmetric = np.allclose(A1.toarray(), A1.toarray().T, atol=1e-6)
print("A1_is_symmetric:", A1_is_symmetric, "\n", A1.toarray().round(4))

uh1[:] = spsolve(A1, F1)
print("uh1:\n", uh1)

A1_is_symmetric: True 
 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  4.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  4. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  4.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 

在 $A$ 和 $F$ 中，只对区域内部自由度对应的子矩阵和子向量进行求解，这样会改变原始离散系统的规模（这种方式的矩阵 A 不一定是对称的）

In [19]:
# Apply Dirichlet boundary conditions
uh2 = space.function()
isBdDof = space.is_boundary_dof()
ipoints = space.interpolation_points() 
uh2[isBdDof] = pde.dirichlet(ipoints[isBdDof])

F2 = F0
F2[isBdDof] = uh2[isBdDof]

bdIdx = np.zeros(NDof, dtype=np.int_)
bdIdx[isBdDof] = 1
Tbd = spdiags(bdIdx, 0, NDof, NDof) 
T = spdiags(1-bdIdx, 0, NDof, NDof)
A2 = T@A0 + Tbd
A2_is_symmetric1 = np.allclose(A2.toarray(), A2.toarray().T, atol=1e-6)
print("A2_is_symmetric1:", A2_is_symmetric1, "\n", A2.toarray().round(4))

uh2[:] = spsolve(A2, F2)
print("uh2:\n", uh2)

A2_is_symmetric1: False 
 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  4. -1.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0

In [20]:
error = np.sum(np.abs(uh1 - uh2))
print("error:", error)

error: 1.3877787807814457e-16
